In [10]:
import os
from getpass import getpass

# Mount Google Drive
if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')
else:
  print("Google Drive already mounted")

# Git configuration
!git config --global user.name "sammig6i"
!git config --global user.email "swghazzawi@gmail.com"

# Securely input your GitHub personal access token
token = getpass('Enter your GitHub token: ')

# Clone the GitHub repository
!git clone https://{token}@github.com/sammig6i/NBA-Fantasy-Points-Prediction.git

# Copy the notebook to the repository
!cp /content/drive/MyDrive/NBA_Fantasy_Clustering.ipynb /content/NBA-Fantasy-Points-Prediction/notebooks/

# Navigate to the repository directory
%cd /content/NBA-Fantasy-Points-Prediction

# Add the notebook file to the repository
!git add notebooks/NBA_Fantasy_Clustering.ipynb

# Commit the changes
!git commit -m "Add clustering notebook for NBA Fantasy project"

# Push the changes to GitHub securely
!git push https://{token}@github.com/sammig6i/NBA-Fantasy-Points-Prediction.git


Google Drive already mounted
Enter your GitHub token: ··········
fatal: destination path 'NBA-Fantasy-Points-Prediction' already exists and is not an empty directory.
/content/NBA-Fantasy-Points-Prediction
On branch main
Your branch is ahead of 'origin/main' by 5 commits.
  (use "git push" to publish your local commits)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	NBA-Fantasy-Points-Prediction/

nothing added to commit but untracked files present (use "git add" to track)
fatal: unable to access 'https://github.com:sammig6i/NBA-Fantasy-Points-Prediction.git@github.com/sammig6i/NBA-Fantasy-Points-Prediction.git/': URL using bad/illegal format or missing URL
